In [0]:
%run "./00 - Setup"

## Use Kepler library (GPU accelerated using WebGL) as alternative to Folium

In [0]:
%pip install keplergl==0.3.2 --quiet


In [0]:
dbutils.library.restartPython() # <- restart python kernel

#### Load utility class to help us display maps in cells

In [0]:
from src.spatial.viz.kepler import *

## Perform search within a radius

There is no native function to do that like PostGIS ST_DWithin. But it's possible to perform this type of search use a combination of st_buffer and st_intersects.

In [0]:
%sql
SELECT * FROM ${catalog}.${database}.ufo_sightings
WHERE st_intersects(st_buffer(st_geomfromtext(geometry), 0.3), st_point(-97, 29))
LIMIT 10

## Join Airport
### Optimized Spatial Join Approach using H3

Performing a direct spatial join between all airports and UFO sightings would be computationally expensive. Instead, we:

1. H3 Indexing
    - Calculate the H3 hexagon index for both airport and UFO sighting locations
    - Use the parent tile at a lower resolution (coarser granularity) to group nearby features

2. Proximity Filtering
    - Perform an initial join using these parent H3 tiles to establish potential spatial relationships

3. Precise Distance Calculation

    - Apply ST_DistanceSphere (or equivalent spatial function) only to candidate pairs from the H3 join
    - Filter results by actual geographic distance threshold

This hierarchical approach reduces computational complexity from O(n²) to O(n log n) while maintaining accuracy through final distance verification.



In [0]:
from pyspark.sql.functions import col, expr, row_number
from pyspark.sql.window import Window

ufo_sightings_df = spark.read.table(f"{catalog}.{database}.ufo_sightings")
airports_df = spark.read.table(f"{catalog}.{database}.airports")

search_radius_km = 10000  # Example: 10 km radius

# Ensure both DataFrames have H3 cell IDs
ufo_sightings_df = ufo_sightings_df.withColumn("h3_ufo", expr("H3_POINTASH3(geometry, 9)")) \
  .withColumn("h3_res6_cell_id", expr("H3_POINTASH3(geometry, 6)")) \
  .withColumn("ufo_parent", expr("H3_TOPARENT(h3_ufo, 5)")).alias("ufo")
airports_df = airports_df.withColumnRenamed("id", "airport_id") \
  .withColumn("h3_airport", expr("H3_POINTASH3(geometry, 9)")) \
  .withColumn("airport_parent", expr("H3_TOPARENT(h3_airport, 5)")).alias("airports")

joined_df = ufo_sightings_df \
  .join(airports_df, expr(f"ufo_parent == airport_parent AND st_distancesphere(ufo.geometry, airports.geometry) <= {search_radius_km}"), "left") \
  .withColumn("distance_airport", expr("st_distancesphere(ufo.geometry, airports.geometry)")) \
  .withColumn("line_string", expr("st_astext(st_makeline(array(st_setsrid(st_geomfromtext(ufo.geometry), 4326), st_geomfromgeojson(airports.geometry))))"))

In [0]:
joined_df.where("city = 'dallas'").display()


![Alt text](../data/h3_to_h3_parent_join.png)


In [0]:
map_render_dfMapItems(
      DFMapItem(joined_df.where("city = 'dallas'").select(F.expr("h3_h3tostring(h3_airport) as h3_airport"), F.expr("h3_h3tostring(h3_ufo) as h3_ufo"), F.expr("h3_h3tostring(airport_parent) as airport_parent"), F.expr("h3_h3tostring(ufo_parent) as ufo_parent"), "name", F.expr("ufo.geometry as ufo_geometry"), F.expr("airports.geometry as airport_geometry"), "line_string"), "h3_airport", RENDER_TYPE.H3_STR, exclude_cols=[])
)

### Only keep the airport closer to the sighting

In [0]:
from pyspark.sql.functions import col, expr, row_number, when
from pyspark.sql.window import Window

# Window specification to get the closest airport
window_spec = Window.partitionBy("id").orderBy(col("distance_airport"))

# Add row number to each row within the window
ranked_df = joined_df.withColumn("row_number", row_number().over(window_spec))

# Filter to keep only the closest airport
filtered_ufo_sightings_df = ranked_df.filter(col("row_number") == 1).drop("row_number", "properties")

filtered_ufo_sightings_df = filtered_ufo_sightings_df.select("ufo.*", "airport_id").withColumn(
    "airport_closed_by",
    when(col("airport_id").isNotNull(), True).otherwise(False)
)

In [0]:
filtered_ufo_sightings_df.display()

## Join with major cities (rural vs urban settings)

In [0]:

cities_df = spark.read.table(f"{catalog}.{database}.major_cities")

search_radius_km = 100  # Example: 5 km radius

# Ensure both DataFrames have H3 cell IDs
cities_df = cities_df.withColumnRenamed("id", "city_id") \
  .withColumn("h3_city", expr("H3_POINTASH3(geometry, 9)")) \
  .withColumn("city_parent", expr("H3_TOPARENT(h3_city, 5)")).alias("cities")

joined_df = filtered_ufo_sightings_df \
  .join(cities_df, expr(f"ufo_parent == city_parent AND st_distance(ufo.geometry, cities.geometry) <= {search_radius_km}"), "left") \
  .withColumn("distance_city", expr("st_distance(ufo.geometry, cities.geometry)"))

In [0]:
# Window specification to get the closest airport
window_spec = Window.partitionBy("id").orderBy(col("distance_city"))

# Add row number to each row within the window
ranked_df = joined_df.withColumn("row_number", row_number().over(window_spec))

# Filter to keep only the closest airport
filtered_ufo_sightings_df = ranked_df.filter(col("row_number") == 1).drop("row_number", "properties")

filtered_ufo_sightings_df = filtered_ufo_sightings_df.select("ufo.*", "airport_id", "airport_closed_by", "city_id").withColumn(
    "urban",
    when(col("city_id").isNotNull(), True).otherwise(False)
)

In [0]:
filtered_ufo_sightings_df.display()

In [0]:
map_render_dfMapItems(
  DFMapItem(filtered_ufo_sightings_df.limit(1000).select(F.expr("h3_h3tostring(h3_ufo) as ufo_h3")), "ufo_h3", RENDER_TYPE.H3_STR, exclude_cols=[]),
    DFMapItem(filtered_ufo_sightings_df.limit(1000).select(F.expr("h3_h3tostring(ufo_parent) as ufo_parent")), "ufo_parent", RENDER_TYPE.H3_STR, exclude_cols=[])
)

## Spatial Join on counties in SQL

Use coarse resolution for display

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW valid_counties AS
  SELECT
    *
  FROM ${catalog}.${database}.counties
  WHERE ST_IsValid(geometry);

CREATE OR REPLACE TEMPORARY VIEW h3_tessellation AS
  SELECT
    *,
    EXPLODE(H3_TESSELLATEASWKB(geometry, 6)) AS h3
  FROM valid_counties;

SELECT count(*) FROM h3_tessellation;

In [0]:
spark.table("h3_tessellation").selectExpr("ST_ASTEXT(ST_GEOMFROMWKB(h3.chip)) as chip", "county", "population").where("state == 'TX'").display()

In [0]:
map_render_dfMapItems(
  DFMapItem(spark.table("h3_tessellation").selectExpr("h3.cellid as index", "county", "population").limit(400000), "index", RENDER_TYPE.H3_INT, exclude_cols=[]),
)

In [0]:
map_render(spark.table("h3_tessellation").selectExpr("ST_ASTEXT(ST_GEOMFROMWKB(h3.chip)) as chip", "county", "population").where("state == 'TX'"), "chip")

Use same H3 resolution than for UFO Sightings

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW valid_counties AS
  SELECT
    *
  FROM ${catalog}.${database}.counties
  WHERE ST_IsValid(geometry);

CREATE OR REPLACE TEMPORARY VIEW h3_counties AS
  SELECT
    county, population, id as county_id,
    EXPLODE(H3_TESSELLATEASWKB(geometry, 6)) AS h3
  FROM valid_counties;

SELECT * FROM h3_counties;

In [0]:
filtered_ufo_sightings_df.createOrReplaceTempView("filtered_ufo_sightings")

In [0]:
%sql

CREATE OR REPLACE TABLE ${catalog}.${database}.enriched_ufo_sightings
SELECT
  id, datetime, shape, duration_seconds, comments, city, state, latitude, longitude, geometry, airport_id, airport_closed_by, urban, city_id, county, population, county_id, '' as reason
FROM filtered_ufo_sightings u
  INNER JOIN h3_counties c
  ON u.h3_res6_cell_id = c.h3.cellid
WHERE
  c.h3.core OR ST_CONTAINS(c.h3.chip, u.geometry)

In [0]:
%sql
ALTER TABLE ${catalog}.${database}.enriched_ufo_sightings
ADD CONSTRAINT fk_city_id
FOREIGN KEY (city_id) REFERENCES ${catalog}.${database}.major_cities(id);

ALTER TABLE ${catalog}.${database}.enriched_ufo_sightings
ADD CONSTRAINT fk_airport_id
FOREIGN KEY (airport_id) REFERENCES ${catalog}.${database}.airports(id);

ALTER TABLE ${catalog}.${database}.enriched_ufo_sightings
ADD CONSTRAINT fk_county_id
FOREIGN KEY (county_id) REFERENCES ${catalog}.${database}.counties(id);

## Compute stats

In [0]:
%sql
SELECT 
  urban,
  COUNT(*) AS count,
  (COUNT(*) * 100.0 / SUM(COUNT(*)) OVER ()) AS percentage
FROM ${catalog}.${database}.enriched_ufo_sightings
GROUP BY urban;

In [0]:
%sql
SELECT 
  airport_closed_by,
  COUNT(*) AS count,
  (COUNT(*) * 100.0 / SUM(COUNT(*)) OVER ()) AS percentage
FROM ${catalog}.${database}.enriched_ufo_sightings
GROUP BY airport_closed_by;

**Is there a correlation between population and number of sightings?**

Most populous counties seems to have observe the most UFO sightings.

In [0]:
%sql
WITH sightings_count AS (
    SELECT 
        county_id, 
        COUNT(*) AS num_sightings
    FROM 
        ${catalog}.${database}.enriched_ufo_sightings
    GROUP BY 
        county_id
)
SELECT 
    int(sc.num_sightings),
    int(c.population)
FROM 
    sightings_count sc
JOIN 
    ${catalog}.${database}.counties c
ON 
    sc.county_id = c.id
WHERE sc.num_sightings > 100;



![Alt text](../data/population_correlation.png)


## Detect clusters with hull

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW sightings_hull AS (
  SELECT 
    st_aswkt(st_convexhull(st_makeline(collect_list(ST_GeomFromText(geometry))))) AS g,
    count(county) as total
  FROM ${catalog}.${database}.enriched_ufo_sightings
  GROUP BY county, state
);

SELECT * FROM sightings_hull
WHERE g LIKE 'POLYGON%';

In [0]:
map_render(spark.table("sightings_hull").filter("g LIKE 'POLYGON%'"), "g")


Using Kepler, show sightings with gradient coloring on the duration field.

In [0]:
map_render(spark.table(f"{catalog}.{database}.enriched_ufo_sightings"), "geometry")

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW sightings_county AS (
  SELECT 
    county_id,
    count(county_id) as total
  FROM ${catalog}.${database}.enriched_ufo_sightings
  GROUP BY county_id
);

SELECT * FROM sightings_county s
 JOIN ${catalog}.${database}.counties c ON c.id = s.county_id;

In [0]:
map_render(spark.sql(f"SELECT * FROM sightings_county s JOIN {catalog}.{database}.counties c ON c.id = s.county_id"), "geometry")

## Use GenAI to come up with possible explanations

In [0]:
%sql
CREATE OR REPLACE FUNCTION explain_ufo(description STRING, location STRING, datetime STRING)
  RETURNS STRING
  RETURN ai_query(
    'databricks-claude-3-7-sonnet',
    "can you explain what would be the best explanation for this ufo sightseeing: " || description || " that happened at " || datetime || " at " || location);

SELECT explain_ufo("silent red /orange mass of energy floated by three of us ", "10/10/1998 20:30", "nyc/westchester area");

In [0]:
%sql
-- Create a temporary view with the updated values
CREATE OR REPLACE TEMP VIEW updated_ufo_sightings AS
SELECT *,
       explain_ufo(comments, datetime, city || ' ' || state) AS new_reason
FROM ${catalog}.${database}.enriched_ufo_sightings
LIMIT 10;

-- Perform the update using the MERGE statement
MERGE INTO ${catalog}.${database}.enriched_ufo_sightings AS target
USING updated_ufo_sightings AS source
ON target.id = source.id
WHEN MATCHED THEN
  UPDATE SET target.reason = source.new_reason;

In [0]:
%sql
SELECT *,
       explain_ufo(comments, datetime, city || ' ' || state) AS new_reason
FROM ${catalog}.${database}.enriched_ufo_sightings
LIMIT 10

In [0]:
%sql
WITH text AS (
  SELECT "silent red /orange mass of energy floated by three of us in western North Carolina in the 60s" AS description
)
SELECT *,
  ai_query(
    'databricks-meta-llama-3-1-8b-instruct',
    "can you explain what would be the best explanation for this ufo sighting: " || description
  )
FROM text;

In [0]:
%sql
WITH text AS (
  SELECT "silent red /orange mass of energy floated by three of us in western North Carolina in the 60s" AS description
)
SELECT *,
  ai_query(
    'databricks-llama-4-maverick',
    "can you explain what would be the best explanation for this ufo sighting: " || description
  )
FROM text;

In [0]:
%sql
WITH text AS (
  SELECT "silent red /orange mass of energy floated by three of us in western North Carolina in the 60s" AS description
)
SELECT *,
  ai_query(
    'databricks-claude-3-7-sonnet',
    "can you explain what would be the best explanation for this ufo sighting: " || description
  )
FROM text;